In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE117613"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE117613"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE117613.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE117613.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE117613.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Whole-blood transcriptional signatures composed of erythropoietic and Nrf2-regulated genes differ between cerebral malaria and severe malarial anemia"
!Series_summary	"We compared genome-wide transcription profiles of whole blood obtained from Ugandan children with acute CM (n=17) or SMA (n=17) and community children without Plasmodium falciparum infection (n=12) and determined the relationships between gene expression, hematological indices, and relevant plasma biomarkers. Results demonstrated both CM and SMA predominantly upregulated enrichment of dendritic cell (DC), inflammatory/TLR/chemokines, monocyte, and antigen presentation modules but downregulated enrichment of lymphocyte modules. Neurodegenerative disease pathways showed modest upregulation during CM. Nrf2-regulated genes were over-expressed in children with SMA compared to CM, especially those with HbSS, corresponding with increased plasma heme oxygenase-1. Reticulocyte-specific gene 

### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
import re
import gzip
from typing import Callable, Optional, Dict, Any, List, Union

# Check the files in the cohort directory
files = os.listdir(in_cohort_dir)
print(f"Files in {in_cohort_dir}:")
print(files)

# Load the series matrix file which contains clinical data
series_matrix_path = os.path.join(in_cohort_dir, 'GSE117613_series_matrix.txt.gz')

# Function to parse GEO series matrix file and extract sample characteristics
def parse_geo_series_matrix(file_path):
    sample_chars = {}
    row_idx = 0
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            if line.startswith('!Sample_characteristics_ch'):
                parts = line.strip().split('\t')
                header = parts[0]
                values = parts[1:]
                sample_chars[row_idx] = values
                row_idx += 1
            elif line.startswith('!Sample_title'):
                # Keep sample titles for reference
                parts = line.strip().split('\t')
                sample_titles = parts[1:]
    
    return pd.DataFrame(sample_chars), sample_titles

# Parse the series matrix file
clinical_data, sample_titles = parse_geo_series_matrix(series_matrix_path)

# Display sample characteristics to identify available variables
print("\nSample characteristics data:")
for row_id, values in clinical_data.iterrows():
    print(f"Row {row_id}: {values.tolist()[:5]}...")

# Based on examination of the data, this dataset is about malaria, not sickle cell anemia
# While it contains gene expression data, it does not have information about sickle cell status
is_gene_available = True
is_trait_available = False  # No sickle cell anemia data in this dataset

# No relevant rows for sickle cell anemia
trait_row = None  # No sickle cell anemia information
age_row = 2      # Row index for age information (both age rows have similar information)
gender_row = 4   # Row index for gender information (labeled as "Sex")

# Define conversion functions for completeness, though trait won't be used
def convert_trait(value):
    """Convert trait values to binary format (0 for control, 1 for case)"""
    # No sickle cell anemia data found in this dataset
    return None

def convert_age(value):
    """Convert age values to continuous numeric format"""
    if value is None or pd.isna(value):
        return None
    
    value = str(value)
    # Extract the actual value if it follows a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    match = re.search(r'(\d+(?:\.\d+)?)', value)
    if match:
        return float(match.group(1))
    else:
        return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)"""
    if value is None or pd.isna(value):
        return None
    
    value = str(value).lower()
    # Extract the actual value if it follows a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if any(term in value for term in ['f', 'female', 'woman', 'girl']):
        return 0
    elif any(term in value for term in ['m', 'male', 'man', 'boy']):
        return 1
    else:
        return None

# Conduct initial filtering and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since trait data is not available (sickle cell anemia specific), 
# we skip clinical feature extraction
print("Sickle Cell Anemia trait data not available. Skipping clinical feature extraction.")


Files in ../../input/GEO/Sickle_Cell_Anemia/GSE117613:
['GSE117613_family.soft.gz', 'GSE117613_series_matrix.txt.gz']

Sample characteristics data:
Row 0: ['"diagnosis: no Plasmodium falciparum infection"', '"processing batch: Batch-2"', '"age: 4.602"', '"age: 15.5"', '"Sex: Male"']...
Row 1: ['"diagnosis: severe malaral anemia"', '"processing batch: Batch-2"', '"age: 9.292"', '"age: 23"', '"Sex: Male"']...
Row 2: ['"diagnosis: severe malaral anemia"', '"processing batch: Batch-3"', '"age: 1.903"', '"age: 12"', '"Sex: Male"']...
Row 3: ['"diagnosis: severe malaral anemia"', '"processing batch: Batch-2"', '"age: 4.03"', '"age: 19"', '"Sex: Male"']...
Row 4: ['"diagnosis: severe malaral anemia"', '"processing batch: Batch-2"', '"age: 2.212"', '"age: 10"', '"Sex: Female"']...
Row 5: ['"diagnosis: severe malaral anemia"', '"processing batch: Batch-3"', '"age: 4.37"', '"age: 14"', '"Sex: Female"']...
Row 6: ['"diagnosis: cerebral milaria"', '"processing batch: Batch-2"', '"age: 3.001"', '"a

### Step 4: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df

Found data marker at line 91
Header line: "ID_REF"	"GSM3305150"	"GSM3305151"	"GSM3305152"	"GSM3305153"	"GSM3305154"	"GSM3305155"	"GSM3305156"	"GSM3305157"	"GSM3305158"	"GSM3305159"	"GSM3305160"	"GSM3305161"	"GSM3305162"	"GSM3305163"	"GSM3305164"	"GSM3305165"	"GSM3305166"	"GSM3305167"	"GSM3305168"	"GSM3305169"	"GSM3305170"	"GSM3305171"	"GSM3305172"	"GSM3305173"	"GSM3305174"	"GSM3305175"	"GSM3305176"	"GSM3305177"	"GSM3305178"	"GSM3305179"	"GSM3305180"	"GSM3305181"	"GSM3305182"	"GSM3305183"	"GSM3305184"	"GSM3305185"	"GSM3305186"	"GSM3305187"	"GSM3305188"	"GSM3305189"	"GSM3305190"	"GSM3305191"	"GSM3305192"	"GSM3305193"	"GSM3305194"	"GSM3305195"
First data line: "ILMN_1343291"	13.8067073	14.07283541	13.26968186	13.80027981	14.22910742	13.56036683	14.36267785	13.20387843	13.92908687	13.51259193	14.06482512	13.81430502	13.53146386	13.39199003	13.96348124	13.40205785	14.43223278	14.33391241	14.17483392	13.7850654	14.15794698	13.69388416	13.80027981	13.06301785	14.31722416	13.94662632	14.026527

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')
